In [16]:
#ссылка на мой репозиторий в git hub: https://github.com/angelica-t/HSE-Machine-Learning
import pandas as pd
import os
import json
import numpy as np
from pandas.io.json import json_normalize


#some_data=data.sample(frac=0.1, random_state=2)
#print(some_data.dtypes)
data = pd.read_csv("./train.csv", low_memory=False)
df1 = data.sample(frac=0.1, random_state=2)
df_s = data.sample(frac=0.01, random_state=2)

df1.to_csv(r'10%_data.csv', index = False)
df_s.to_csv(r'1%_data.csv', index = False)

In [17]:
# эта часть кода взята с kaggle
def load_df(csv_path='./10%_data.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [18]:
df_train = load_df()
df_train


Loaded 10%_data.csv. Shape: (90365, 54)


,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,...,trafficSource.medium,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.keyword,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd
0,Social,20161129,324272247457982354,324272247457982354_1480453021,Not Socially Engaged,1480453021,1,1480453021,Chrome,not available in demo dataset,...,referral,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,20170317,5322552539294131300,5322552539294131300_1489817299,Not Socially Engaged,1489817299,1,1489817299,Chrome,not available in demo dataset,...,organic,not available in demo dataset,(not provided),NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Social,20161003,860949384581209567,860949384581209567_1475520229,Not Socially Engaged,1475520229,1,1475520229,Chrome,not available in demo dataset,...,referral,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Social,20161113,6907002564702847740,6907002564702847740_1479085291,Not Socially Engaged,1479085291,1,1479085291,Safari,not available in demo dataset,...,referral,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,20170731,5482421667155283216,5482421667155283216_1501528076,Not Socially Engaged,1501528076,1,1501528076,Chrome,not available in demo dataset,...,organic,not available in demo dataset,(not provided),NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90360,Organic Search,20161227,1424669229499013118,1424669229499013118_1482890199,Not Socially Engaged,1482890199,2,1482890199,Chrome,not available in demo dataset,...,organic,not available in demo dataset,(not provided),True,NaN,NaN,NaN,NaN,NaN,NaN
90361,Referral,20170314,7324846162851204153,7324846162851204153_1489495292,Not Socially Engaged,1489495292,1,1489495292,Chrome,not available in demo dataset,...,referral,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90362,Direct,20160824,0971946123643721899,0971946123643721899_1472033004,Not Socially Engaged,1472033004,1,1472033004,Safari,not available in demo dataset,...,(none),not available in demo dataset,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
90363,Organic Search,20160921,7280055078726300080,7280055078726300080_1474475417,Not Socially Engaged,1474475417,2,1474475417,Chrome,not available in demo dataset,...,organic,not available in demo dataset,(not provided),True,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_train

In [20]:
# удаляем ненужные столбцы
#df_train.drop(df_train.columns[[9, 10, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 27, 28, 29, 30, 33, 34, 39, 41, 42, 45, 46, 47, 48, 49, 50, 51, 51, 53]], axis = 1, inplace = True) 
df_train=df_train.drop(['geoNetwork.latitude' ,'trafficSource.adwordsClickInfo.adNetworkType'],axis = 1)
df_train

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.operatingSystem,...,geoNetwork.subContinent,geoNetwork.country,geoNetwork.networkDomain,totals.visits,totals.hits,totals.pageviews,totals.newVisits,totals.transactionRevenue,trafficSource.source,trafficSource.medium
0,Social,20161129,324272247457982354,324272247457982354_1480453021,Not Socially Engaged,1480453021,1,1480453021,Chrome,Windows,...,Southern Europe,Portugal,vodafone.pt,1,3,3,1,NaN,youtube.com,referral
1,Organic Search,20170317,5322552539294131300,5322552539294131300_1489817299,Not Socially Engaged,1489817299,1,1489817299,Chrome,Windows,...,Eastern Asia,Hong Kong,netvigator.com,1,1,1,1,NaN,google,organic
2,Social,20161003,860949384581209567,860949384581209567_1475520229,Not Socially Engaged,1475520229,1,1475520229,Chrome,Windows,...,Eastern Europe,Russia,avangarddsl.ru,1,4,3,1,NaN,youtube.com,referral
3,Social,20161113,6907002564702847740,6907002564702847740_1479085291,Not Socially Engaged,1479085291,1,1479085291,Safari,Macintosh,...,Southeast Asia,Thailand,3bb.co.th,1,1,1,1,NaN,youtube.com,referral
4,Organic Search,20170731,5482421667155283216,5482421667155283216_1501528076,Not Socially Engaged,1501528076,1,1501528076,Chrome,Windows,...,Northern America,United States,express-scripts.com,1,3,3,1,NaN,google,organic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90360,Organic Search,20161227,1424669229499013118,1424669229499013118_1482890199,Not Socially Engaged,1482890199,2,1482890199,Chrome,Android,...,Western Europe,Germany,(not set),1,3,3,NaN,NaN,google,organic
90361,Referral,20170314,7324846162851204153,7324846162851204153_1489495292,Not Socially Engaged,1489495292,1,1489495292,Chrome,Windows,...,Southern Asia,India,(not set),1,1,1,1,NaN,analytics.google.com,referral
90362,Direct,20160824,0971946123643721899,0971946123643721899_1472033004,Not Socially Engaged,1472033004,1,1472033004,Safari,Macintosh,...,Northern Europe,Finland,nebulazone.fi,1,1,1,1,NaN,(direct),(none)
90363,Organic Search,20160921,7280055078726300080,7280055078726300080_1474475417,Not Socially Engaged,1474475417,2,1474475417,Chrome,Windows,...,South America,Peru,sitel.com.pe,1,13,11,NaN,NaN,google,organic


In [29]:
# делаем категории из данных и приводим дату в нормальный формат
from datetime import datetime
df_train['visitStartTime']=pd.to_datetime(df_train['visitStartTime'], unit='s')
df_train['totals.pageviews']=df_train['totals.pageviews'].fillna(0.0).astype(float)
df_train['totals.transactionRevenue']=df_train['totals.transactionRevenue'].fillna(0.0).astype(float)
df_train['totals.newVisits']=df_train['totals.newVisits'].fillna(0.0).astype(float)
browser=df_train['device.browser'].astype('category')
operSystem=df_train['device.operatingSystem'].astype('category')
subcont=df_train['geoNetwork.subContinent'].astype('category')
channel=df_train['channelGrouping'].astype('category')
engagedOrNot=df_train['socialEngagementType'].astype('category')

In [30]:
df_train.dtypes

channelGrouping                      object
date                                  int64
fullVisitorId                        object
sessionId                            object
socialEngagementType                 object
visitId                               int64
visitNumber                           int64
visitStartTime               datetime64[ns]
device.browser                       object
device.operatingSystem               object
device.isMobile                        bool
device.deviceCategory                object
geoNetwork.continent                 object
geoNetwork.subContinent              object
geoNetwork.country                   object
geoNetwork.networkDomain             object
totals.visits                        object
totals.hits                          object
totals.pageviews                    float64
totals.newVisits                    float64
totals.transactionRevenue           float64
trafficSource.source                 object
trafficSource.medium            

In [31]:
# ГРУППИРОВКА И ФУНКЦИИ
from math import log1p
def agg(group): # функция, в которой считаются все показатели
    freeday=0
    daytime=0
    revenueForUser=log1p((group['totals.transactionRevenue']).sum())
    pageViewsForUser=sum(group['totals.pageviews'])
    numberOfDevices=group['device.operatingSystem'].nunique()
    for index, row in group.iterrows(): # пробегаемся по датам в каждой группе 
        if row['visitStartTime'].strftime('%A')=='Saturday' or row['visitStartTime'].strftime('%A')=='Sunday':
            freeday=1 #выходной день или нет
        if 5<=row['visitStartTime'].hour<=21:
            daytime=1 #дневное или ночное время
            
    return freeday, daytime, revenueForUser, pageViewsForUser, numberOfDevices
def func(group): #функция, в которой все посчитанные показатели собираются в один массив
    freeday,daytime, revenueForUser,pageViewsForUser,numberOfDevices=agg(group)
    d = {}
    d['Weekend or not'] = freeday
    d['Daytime or not'] = daytime
    d['Revenue']=revenueForUser
    d['Page views']=pageViewsForUser
    d['Number of devices']=numberOfDevices
    return pd.Series(d, index=['Weekend or not','Daytime or not','Revenue','Page views','Number of devices'])
groupedData = df_train.groupby('fullVisitorId').apply(func)
groupedData= groupedData.reset_index()
groupedData['Browser']=browser.cat.codes
groupedData['Operating system']=operSystem.cat.codes
groupedData['Part of the world']=subcont.cat.codes
groupedData['Channel types']=channel.cat.codes
groupedData['Socially engaged']=engagedOrNot.cat.codes
groupedData

,fullVisitorId,Weekend or not,Daytime or not,Revenue,Page views,Number of devices,Browser,Operating system,Part of the world,Channel types,Socially engaged
0,0000010278554503158,0.0,1.0,0.0,1.0,1.0,6,11,19,7,0
1,000005103959234087,1.0,0.0,0.0,1.0,1.0,6,11,6,4,0
2,0000253133284033056,0.0,1.0,0.0,1.0,1.0,6,11,7,7,0
3,0000436683523507380,0.0,1.0,0.0,0.0,1.0,26,7,16,7,0
4,0000496879625382660,0.0,1.0,0.0,1.0,1.0,6,11,12,4,0
...,...,...,...,...,...,...,...,...,...,...,...
85816,9999607385222235728,0.0,1.0,0.0,1.0,1.0,6,7,12,4,0
85817,999961560906226745,0.0,1.0,0.0,1.0,1.0,6,11,15,4,0
85818,9999761280717362235,0.0,1.0,0.0,1.0,1.0,6,1,6,4,0
85819,9999774833256469931,0.0,0.0,0.0,1.0,1.0,6,7,12,2,0


In [32]:
#  бустинг
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
X=groupedData.drop('Revenue',axis=1) # исключаем из данных зависимую переменную
y=groupedData['Revenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
# делаем настройку гиперпараметров
# типы функций потерь
loss = ['ls', 'lad', 'huber']

# количество деревьев
n_estimators = [100, 500, 900, 1100, 1500]

# максимальная глубина
max_depth = [2, 3, 5, 10, 15]
min_samples_leaf = [1, 2, 4, 6, 8]
min_samples_split = [2, 4, 6, 10]
max_features = ['auto', 'sqrt', 'log2', None]

# все гиперпараметры в один массив
hyperparameter_grid = {'loss': loss,
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'max_features': max_features}

gb = GradientBoostingRegressor(random_state=2)
grid_rf = GridSearchCV(estimator=gb,
                       param_grid=hyperparameter_grid,
                       scoring='neg_mean_squared_error'
                       )

grid_rf.fit(X_train, y_train)
# выбираем лучшую модель
best_model = grid_rf.best_estimator_
# предсказываем
y_pred = best_model.predict(X_test)

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import confusion_matrix, classification_report
#  MSE
mse_test = MSE(y_test, y_pred)
#  RMSE
rmse_test = mse_test**(1/2)
from sklearn.model_selection import cross_val_score
#  5-fold cross-validation scores: cv_scores
cv_scores = cross_val_score(best_model, X,y,cv=5)
print('Test set RMSE of gb: {:.3f}'.format(rmse_test))
print('5-fold cross-validation scores')
print(cv_scores)
print("Average 5-Fold CV Score")
print(np.mean(cv_scores))

C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


KeyboardInterrupt: 

In [ ]:
# Случайный лес
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=2)
# определяем гиперпараметры леса
params_rf = {
             'n_estimators': [100, 350, 500],
             'max_features': ['log2', 'auto', 'sqrt'],
             'min_samples_leaf': [2, 10, 30], 
             }
from sklearn.model_selection import  GridSearchCV
grid_rf = GridSearchCV(estimator=rf,
                       param_grid=params_rf,
                       scoring='neg_mean_squared_error',
                       verbose=1,
                       n_jobs=-1)
grid_rf.fit(X_train, y_train)
# выбираем лучшую модель
best_model = grid_rf.best_estimator_
# предсказываем
y_pred = best_model.predict(X_test)
#  RMSE
rmse_test = MSE(y_test, y_pred)**(1/2)
print('Test set RMSE of rf: {:.2f}'.format(rmse_test))
# 5-fold cross-validation scores: cv_scores
cv_scores = cross_val_score(best_model, X,y,cv=5)
print('5-fold cross-validation scores')
print(cv_scores)
print("Average 5-Fold CV Score")
print(np.mean(cv_scores))

In [ ]:
# SVM
from sklearn.svm import SVR
#SVMreg = SVR(kernel='rbf')
parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.5, 10],'gamma': [1e-7, 1e-4],'epsilon':[0.1,0.2,0.5,0.3]}
from sklearn.metrics import make_scorer
scorer = make_scorer(mean_squared_error, greater_is_better=False)
svr_gs = GridSearchCV(SVR(epsilon = 0.01), parameters, scoring=scorer)
svr_gs.fit(X,y)
# выбираем лучшую модель
best_model = GridSearchCV.best_estimator_
y_pred = best_model.predict(X_test)
#SVMreg.fit()
print('SVM Score:')
print(SVMreg.score(X_train, y_train))
print(SVMreg.score(X_test, y_test))
# 5-fold cross-validation scores: cv_scores
cv_scores = cross_val_score(best_model, X,y,cv=5)

print('5-fold cross-validation scores')
print(cv_scores)
print("Average 5-Fold CV Score")
print(np.mean(cv_scores))

In [146]:
# линейная регрессия
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
# настраиваем гиперпараметры
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space}
reg = LinearRegression()
logreg_cv = GridSearchCV(logreg, param_grid)
logreg_cv.fit(X,y)
#reg.fit(X_train,y_train)
# выбираем лучшую модель
best_model = GridSearchCV.best_estimator_
y_pred = best_model.predict(X_test)

# R^2 and RMSE
print("R^2: {}".format(reg.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print("Root Mean Squared Error: {}".format(rmse))
#  5-fold cross-validation scores: cv_scores
cv_scores = cross_val_score(SVMreg, X,y,cv=5)

print('5-fold cross-validation scores')
print(cv_scores)
print("Average 5-Fold CV Score")
print(np.mean(cv_scores))

R^2: -3.795865853239988e-05
Root Mean Squared Error: 2.1702079916435753


C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarni

5-fold cross-validation scores
[-0.00434687  0.00253511 -0.00377188 -0.00049322 -0.00259551]
Average 5-Fold CV Score
-0.0017344763386563321
